In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.document_loaders import TextLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 1500,
#     chunk_overlap = 100,
#     separators=['\n\n', '\n']
# ) 

# text_path = './documents/real_estate_tax.txt'

# loader = TextLoader(text_path)
# document_list = loader.load_and_split(text_splitter)

# embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

# vector_store = Chroma.from_documents(
#     documents=document_list,
#     embedding=embeddings,
#     collection_name = 'real_estate_tax',
#     persist_directory = './real_estate_tax_collection'
# )

In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str # 사용자 질문
    answer: str # 세율
    tax_base_equation: str # 과세표준 계산 수식 
    tax_deduction: str # 공제액 
    market_ratio: str # 공정시장가액비율
    tax_base: str # 과세표준 계산
    
graph_builder = StateGraph(AgentState)

In [4]:
from langchain_chroma import Chroma 
from langchain_openai import OpenAIEmbeddings 

embedding_function = OpenAIEmbeddings(model= 'text-embedding-3-large')
vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'real_estate_tax',
    persist_directory='./real_estate_tax_collection'
)

In [5]:
retriever = vector_store.as_retriever(search_kwargs = {'k': 3})

In [6]:
query = '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'

In [ ]:
# retriever.invoke(query)

In [8]:
from langchain_openai import ChatOpenAI 
from langchain import hub
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = hub.pull('rlm/rag-prompt')

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')


/Users/robert/Desktop/langGrahpPrac/.venv/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
tax_base_retriever_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | rag_prompt 
    | llm 
    | StrOutputParser() 
) 

tax_base_equation_prompt = ChatPromptTemplate.from_messages([
    ('system', '사용자의 질문에서 과세표주을 계산하는 방법을 부연설명 없이 수식만 표현해주세요'),
    ('human', '{tax_base_equation_information}')
])

tax_base_equation_chain = (
        {'tax_base_equation_information': RunnablePassthrough()}
        | tax_base_equation_prompt
        | llm 
        | StrOutputParser()
)

tax_base_chain = {'tax_base_equation_information': tax_base_retriever_chain} | tax_base_equation_chain

def get_tax_base_equation(state: AgentState): 
    tax_base_equation_question = '주택에 대한 종합부동산세 계산시 과세표준을 계산하는 방법을 알려주세요'
    tax_base_equation = tax_base_chain.invoke(tax_base_equation_question)
    return {'tax_base_equation': tax_base_equation}



In [ ]:
# get_tax_base_equation({})
# {'tax_base_equation': '과세표준 = (주택 공시가격 합산 - 공제 금액) × 공정시장가액비율'}

In [11]:
tax_deduction_retriever_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | rag_prompt 
    | llm 
    | StrOutputParser() 
) 

def get_tax_deduction(state: AgentState): 
    tax_deduction_question = '주택에 대한 종합부동산세 계산시 공제금액을 알려주세요'
    tax_deduction = tax_deduction_retriever_chain.invoke(tax_deduction_question)
    return {'tax_deduction': tax_deduction}


In [ ]:
# get_tax_deduction({})
# {'tax_base_equation': '주택에 대한 종합부동산세 계산 시 1세대 1주택자의 경우 공제금액은 12억 원입니다. 법인이나 법인으로 보는 단체의 경우에는 6억 원, 그 외의 경우에는 9억 원이 공제됩니다.'}

In [13]:
from langchain_community.tools import TavilySearchResults
from datetime import date 

tavily_search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

tax_market_ratio_prompt = ChatPromptTemplate.from_messages([
    ('system', '아래 정보를 기반으로 공정시장 가액비율을 계산해주세요\n\nContext:\n{context}'),
    ('human', '{query}')
])

def get_market_ratio(state: AgentState) -> AgentState: 
    query = f'오늘 날짜: {date.today()}에 해당하는 주택 공시가격 공정시장가액비율은 몇 %인가요?'
    context = tavily_search_tool.invoke(query)

    tax_market_ratio_chain = (
        tax_market_ratio_prompt 
        | llm 
        | StrOutputParser()
    )

    market_ratio = tax_market_ratio_chain.invoke({'context': context, 'query': query})

    return {'market_ratio': market_ratio}    


In [ ]:
# get_market_ratio({})
# {'market_ratio': '2025년 주택 공시가격의 공정시장가액비율은 다음과 같습니다:\n\n- 3억원 이하: 43%\n- 6억원 이하: 44%\n- 6억원 초과: 45%\n- 다주택자 및 법인: 60%\n\n이 비율은 주택 공시가격에 따라 적용됩니다.'}

In [15]:
from langchain_core.prompts import PromptTemplate 

tax_base_calculation_prompt = PromptTemplate.from_template("""
주워진 내용을 기반으로 과세표준을 계산해주세요 

과세표준 계산 공식: {tax_base_equation}
공제금액: {tax_deduction}
공정시장가액비율: {market_ratio}
사용자 주택 공시가격 정보: {query}
""")

def calculate_tax_base(state:AgentState): 
    tax_base_equation = state['tax_base_equation']
    tax_deduction = state['tax_deduction']
    market_ratio = state['market_ratio']
    query = state['query']

    tax_base_calculation_chain = (
        tax_base_calculation_prompt
        | llm 
        | StrOutputParser()
    )
    
    tax_base = tax_base_calculation_chain.invoke({
        'tax_base_equation': tax_base_equation,
        'tax_deduction': tax_deduction,
        'market_ratio': market_ratio,
        'query': query
    })

    print(tax_base)

    return {'tax_base': tax_base}

In [16]:
initial_state = {
    'query' : query,
    'tax_base_equation' : '과세표준 = (주택 공시가격 합산 금액 - 공제 금액) x 공정시장가액비율',
    'tax_deduction' : '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원, 일반적인 경우 9억 원입니다. 법인이나 법인으로 보는 단체의 경우는 6억 원이 공제됩니다.',
    'market_ratio': '2025년 주택 공시가격의 공정시장가액비율은 다음과 같습니다:\n\n- 3억원 이하: 43%\n- 6억원 이하: 44%\n- 6억원 초과: 45%\n- 다주택자 및 법인: 60%\n\n이 비율은 주택 공시가격에 따라 적용됩니다.'
}

tax_base = calculate_tax_base(initial_state)
# tax_base

과세표준을 계산하기 위해 주어진 주택 공시가격을 바탕으로 계산을 진행하겠습니다.

사용자가 소유한 주택의 공시가격은 다음과 같습니다:
- 5억 원짜리 집 1채
- 10억 원짜리 집 1채
- 20억 원짜리 집 1채

총 공시가격 합산 금액:
\[ 5억 + 10억 + 20억 = 35억 \]

공제금액은 일반적인 경우 9억 원입니다.

다음으로, 각 주택에 대해 공정시장가액비율을 적용합니다.

1. 5억 원짜리 집: 공시가격이 6억 원 이하이므로 비율은 44%
2. 10억 원짜리 집: 공시가격이 6억 원 초과이므로 비율은 45%
3. 20억 원짜리 집: 공시가격이 6억 원 초과이므로 비율은 45%

하지만, 다주택자의 경우에는 공정시장가액비율이 60%로 적용됩니다. 따라서, 전체 공시가격에 대해 60%를 적용해야 합니다.

과세표준 계산:
\[ \text{과세표준} = (35억 - 9억) \times 0.60 \]
\[ \text{과세표준} = 26억 \times 0.60 \]
\[ \text{과세표준} = 15.6억 \]

따라서, 과세표준은 15.6억 원입니다. 이 과세표준을 바탕으로 종합부동산세를 계산하게 됩니다.


In [17]:
tax_rate_calculation_prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 종합부동산세 계산 전문가입니다. 아래 문서를 참고해서 사용자의 질문에 대한 종합부동산세를 계산해주세요\n\n종합부동산세 세율:\n{context}'),
    ('human', '과세표준과 사용자가 소지한 주택의 수가 아래와 같을 때 종합부동산세를 계산해주세요\n\n과세표준: {tax_base}\n주택 수: {query}')
])

def calculate_tax_rate(state: AgentState):
    query = state['query']
    tax_base = state['tax_base'] 

    context = retriever.invoke(query)

    tax_rate_chain = (
        tax_rate_calculation_prompt 
        | llm 
        | StrOutputParser()
    )

    tax_rate = tax_rate_chain.invoke({'context': context, 'query': query, 'tax_base': tax_base})

    return {'answer': tax_rate}


In [ ]:
# calculate_tax_rate(
#     {
#         'query': query,
#         'tax_base': tax_base['tax_base'],
#     }
# )

{'answer': '사용자가 3주택을 소유하고 있으므로, 3주택 이상 소유 시 적용되는 세율을 사용해야 합니다. 주어진 과세표준은 15.6억 원입니다.\n\n3주택 이상 소유한 경우의 세율표는 다음과 같습니다:\n\n- 12억 원 초과 25억 원 이하: 960만 원 + (12억 원을 초과하는 금액의 1천분의 20)\n\n과세표준이 15.6억 원이므로, 12억 원을 초과하는 금액은 3.6억 원입니다.\n\n세액 계산은 다음과 같습니다:\n\\[ \\text{세액} = 960만 원 + (3.6억 원 \\times 0.002) \\]\n\\[ \\text{세액} = 960만 원 + 720만 원 \\]\n\\[ \\text{세액} = 1,680만 원 \\]\n\n따라서, 사용자는 종합부동산세로 1,680만 원을 내야 합니다.'}

In [19]:
graph_builder.add_node('get_tax_base_equation', get_tax_base_equation)
graph_builder.add_node('get_tax_deduction', get_tax_deduction)
graph_builder.add_node('get_market_ratio', get_market_ratio)
graph_builder.add_node('calculate_tax_base', calculate_tax_base)
graph_builder.add_node('calculate_tax_rate', calculate_tax_rate) 

In [20]:
from langgraph.graph import START, END 

graph_builder.add_edge(START, 'get_tax_base_equation')
graph_builder.add_edge(START, 'get_tax_deduction')
graph_builder.add_edge(START, 'get_market_ratio')
graph_builder.add_edge('get_tax_base_equation', 'calculate_tax_base')
graph_builder.add_edge('get_tax_deduction', 'calculate_tax_base')
graph_builder.add_edge('get_market_ratio', 'calculate_tax_base')
graph_builder.add_edge('calculate_tax_base', 'calculate_tax_rate')
graph_builder.add_edge('calculate_tax_rate', END)

In [21]:
graph = graph_builder.compile()

In [ ]:
# from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))
# graph.get_graph().print_ascii()


In [23]:
initial_state = {'query': query}
graph.invoke(initial_state)

사용자가 보유한 주택들의 공시가격과 해당 공정시장가액비율을 바탕으로 과세표준을 계산해 보겠습니다.

사용자의 주택 공시가격:
- 5억 원짜리 집 1채
- 10억 원짜리 집 1채
- 20억 원짜리 집 1채

총 공시가격 합산:
- 5억 + 10억 + 20억 = 35억 원

공제금액:
- 주어진 조건에서 사용자가 1세대 1주택자가 아니라면, 공제금액은 9억 원입니다.

공정시장가액비율:
- 5억 원짜리 집: 44% (3억 초과 6억 이하)
- 10억 원짜리 집: 45% (6억 초과)
- 20억 원짜리 집: 45% (6억 초과)

하지만, 문제에서 제공된 정보에 따르면 여러 채의 집을 소유한 경우에는 각각의 집에 다른 비율을 적용하는 것이 아니라, 최종 합산된 가격에 대해 공정시장가액비율을 적용합니다. 따라서 전체 공시가격에 대한 공정시장가액비율은 45%로 적용됩니다.

과세표준 계산:
\[
\text{과세표준} = (\text{총 공시가격 합산} - \text{공제금액}) \times \text{공정시장가액비율}
\]
\[
= (35억 - 9억) \times 0.45
\]
\[
= 26억 \times 0.45
\]
\[
= 11.7억 원
\]

따라서 과세표준은 11.7억 원입니다. 이 과세표준을 바탕으로 종합부동산세를 계산할 수 있습니다. 종합부동산세율은 별도로 적용됩니다.


{'query': '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?',
 'answer': '사용자가 3채의 주택을 소유하고 있으므로, "납세의무자가 3주택 이상을 소유한 경우"에 해당하는 세율을 적용해야 합니다. 과세표준이 11.7억 원인 경우에 해당하는 세율은 다음과 같습니다:\n\n과세표준: 6억 원 초과 12억 원 이하\n세율: 360만 원 + (6억 원을 초과하는 금액의 1천분의 10)\n\n과세표준이 11.7억 원이므로 6억 원을 초과하는 금액은:\n\n\\[\n11.7억 - 6억 = 5.7억 원\n\\]\n\n따라서 추가 세금은:\n\n\\[\n5.7억 원 \\times 0.001 = 570만 원\n\\]\n\n최종 종합부동산세는:\n\n\\[\n360만 원 + 570만 원 = 930만 원\n\\]\n\n따라서, 사용자가 내야 할 종합부동산세는 930만 원입니다.',
 'tax_base_equation': '과세표준 = (주택의 공시가격 합산 - 공제금액) × 공정시장가액비율',
 'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원, 법인 또는 법인으로 보는 단체는 6억 원, 그 외의 경우는 9억 원입니다. ',
 'market_ratio': '2025년의 주택 공시가격에 대한 공정시장가액비율은 일반적으로 60%로 적용되지만, 1주택자의 경우 예외적으로 43%에서 45%까지 적용됩니다. 3억 원 이하의 주택은 43%, 3억 원 초과 6억 원 이하의 주택은 44%, 6억 원 초과의 주택은 45%의 비율이 적용됩니다.',
 'tax_base': '사용자가 보유한 주택들의 공시가격과 해당 공정시장가액비율을 바탕으로 과세표준을 계산해 보겠습니다.\n\n사용자의 주택 공시가격:\n- 5억 원짜리 집 1채\n- 10억 원짜리 집 1채\n- 20억 원짜리 집 1채\n\n총 공시가격 합산:\n- 5억 + 10억 + 20억 = 35억 원\n\n공제금액: